<a href="https://colab.research.google.com/github/sindhu-2507/Speech_Emotion_Recognition/blob/master/Speech_Sentiment_Analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
! pip install -q kaggle

In [ ]:
from google.colab import files

files.upload()

In [ ]:
! mkdir ~/.kaggle
! cp kaggle.json ~/.kaggle/
! chmod 600 ~/.kaggle/kaggle.json

In [ ]:
!kaggle datasets download -d uwrfkaggler/ravdess-emotional-speech-audio

In [ ]:
!mkdir data
%cd data/
!unzip /content/ravdess-emotional-speech-audio.zip

In [ ]:
import os
import random
import sys
dir_path = '/content/data'
dir_list=os.listdir(dir_path)
dir_list.sort()
print (dir_list)


In [ ]:
dir_list = dir_list[0:24]
print (dir_list)

In [ ]:
# %cd /content
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import os

In [ ]:
import librosa

In [ ]:
pip install --upgrade keras

In [ ]:
#All library imports
%matplotlib inline
import csv
import random
# Preprocessing
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder, StandardScaler
#Keras
# import tensorflow as tf
#This is simply a linear stack of neural network layers, and it's perfect for the type of feed-forward CNN
from keras.models import Sequential, load_model
#keras core layers
from keras.layers import Dense, Dropout, Activation, Flatten
#These are the convolutional layers that will help us efficiently train on the data
from keras.layers import Convolution2D, MaxPooling2D
#Utilities
from keras.utils import np_utils

In [ ]:
# Extracting the features
def extract_features(directory_path,directory_list):
  X = []
  Y = []
  w = 0
  curr_dir = directory_path
  dir_list = directory_list
  for i in dir_list:
    dir_path = os.path.join(curr_dir,i)
    for j in os.listdir(dir_path):
      features = []
      songname = os.path.join(dir_path,j)
      p = j.split("-")
      y, sr = librosa.load(songname)
      mfcc = (librosa.feature.mfcc(y=y, sr=sr,n_mfcc = 40))
      X.append(mfcc)
      Y.append(int(p[2])-1)
      q = len(mfcc[0])
      w = max(w,q)
  return X,Y,w

In [ ]:
path = "/content/data"
X,Y,w = extract_features(path,dir_list)

In [ ]:
# Preprocessing the data
def preprocess(X,Y,w):
  x = np.empty([1440,40,228])
  for i in range(len(X)):
    # print(len(X[0][0]))
    for j in range(len(X[i])):
      q = np.asarray(X[i][j])
      N = w - len(q)
      s = np.pad(q,(0,N),'constant')
      x[i][j] = s
  X = np.asarray(x)
  Y = np.asarray(Y)
  return X,Y

In [ ]:
X,Y = preprocess(X,Y,w)
print(X.shape)
print(Y.shape)

In [ ]:
#Reshaping the X and Y data
def reshape(X):
# print(X.shape)
  X = np.reshape(X,[1440,1,40,228])
  # print(X.shape)
  # print(X[0])
  return X

In [ ]:
X = reshape(X)

In [ ]:
#dividing the data into train,dev and test sets
X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size=0.1, random_state=1)
Y_train = np_utils.to_categorical(y_train,8)
Y_test = np_utils.to_categorical(y_test,8)
print(X_train.shape)
print(Y_train.shape)
print(X_test.shape)
print(Y_test.shape)

In [ ]:
#initialising the model
model = Sequential()

In [ ]:
# best fit till now
model.add(Convolution2D(64, 3 , 3, input_shape=(1,40,228), data_format = 'channels_first'))
model.add(MaxPooling2D(pool_size=(2,2)))
model.add(Activation('relu'))
model.add(Convolution2D(64,3,3))
model.add(MaxPooling2D(pool_size = (2,2)))
model.add(Activation('relu'))
model.add(Flatten())
model.add(Dense(128))
model.add(Activation('relu'))
model.add(Dropout(0.5))
model.add(Dense(8))
model.add(Activation('softmax'))

In [ ]:
model.compile(loss='categorical_crossentropy', metrics=['accuracy'], optimizer='adam')

In [ ]:
model.fit(X_train, Y_train, batch_size=36, epochs=76)

In [ ]:
score = model.evaluate(X_test,Y_test, verbose = 0)
y_pred = model.predict_classes(X_test)
print(score)